In [1]:
import mysql.connector 
from mysql.connector import errorcode
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np
from time import sleep
from random import randint
import json

In [2]:
cnx = mysql.connector.connect(
    host = "ms-proj-db.ckhgdgqfsqpw.us-east-2.rds.amazonaws.com",
    user = "admin",
    passwd = '12345678'
)

In [3]:
cursor = cnx.cursor()

In [4]:
db_name = 'movies'

In [5]:
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

In [6]:
# check to see if the database already 
try:
    cursor.execute("USE {}".format(db_name))

#if the previous line fails because there isn't a db by that name run this line

except mysql.connector.Error as err:
    print("Database {} does not exist.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [7]:
# Create a table for the reviews
DB_NAME = 'movies'

TABLES = {}
TABLES['budget'] = ("""
CREATE TABLE budget
    (
    Lifetime_Gross_Rank FLOAT,
    Movie_Title varchar(200),
    Lifetime_Gross FLOAT,
    Movie_Year FLOAT,
    URL varchar(200),
    PRIMARY KEY (Lifetime_Gross_Rank)
    )
    ENGINE=InnoDB
""")

TABLES['summary'] = ("""
CREATE TABLE summary
    (
    Lifetime_Gross_Rank FLOAT,
    Movie_Title varchar(200),
    Lifetime_Gross FLOAT,
    Domestic_Distributor varchar(200),
    Budget FLOAT,
    MPAA varchar(200),
    Running_Time_Minutes FLOAT,
    Genres varchar(200),
    PRIMARY KEY (Lifetime_Gross_Rank)
    )
    ENGINE=InnoDB
""")

In [8]:
cnx = mysql.connector.connect(
    host = "ms-proj-db.ckhgdgqfsqpw.us-east-2.rds.amazonaws.com",
    user = "admin",
    passwd = "12345678",
    database = DB_NAME
)
cursor = cnx.cursor()

In [9]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")
cursor.close()
cnx.close()

Creating table budget: already exists.
Creating table summary: already exists.


In [11]:
def Pull_Box_Office_Revenues():    
    # Set fixed values
    num_pages = 3
    offset_val = 200
    
    # Set counter variable
    offset_count = 0
    
    # Set dummy variables
    rankings = []
    titles = []
    lifetime_gross = []
    year = []
    URL = []
    
    # Loop through pages
    while offset_count < num_pages * offset_val:
        
        # Set URL for each page equal to response
        if offset_count == 0:
            response = requests.get('https://www.boxofficemojo.com/chart/top_lifetime_gross/?area=XWW')
        else:
            response = requests.get('https://www.boxofficemojo.com/chart/top_lifetime_gross/?offset=' + str(offset_count) + '&area=XWW')
        
        # Pause the loop
        sleep(randint(1,4))
        
        # Parse the content using BeautifulSoup
        Box_office_soup = BeautifulSoup(response.content, 'html.parser')
        
        # Add values to the ranking list
        rank_find_all = Box_office_soup.find_all(class_='a-text-right mojo-header-column mojo-truncate mojo-field-type-rank')
        rankings.extend([r.text for r in rank_find_all])
        
        # Add values to the titles list
        movie_name_find_all = Box_office_soup.find_all(class_='a-text-left mojo-field-type-title')
        titles.extend([r.text for r in movie_name_find_all])
        
        # Add values to the lifetime_gross list
        lifetime_gross_find_all = Box_office_soup.find_all(class_='a-text-right mojo-field-type-money')
        lifetime_gross.extend([int(r.text[1:].replace(',', "")) for r in lifetime_gross_find_all])
        
        # Add values to the years list
        year_find_all = Box_office_soup.find_all(class_='a-text-left mojo-field-type-year')
        year.extend([r.text for r in year_find_all])
        
        # Add URLs to the URL list
        movie_name_find_all = Box_office_soup.find_all(class_='a-text-left mojo-field-type-title')
        URL.extend([('https://www.boxofficemojo.com' + r.a['href']) for r in movie_name_find_all])        
        
        # Move on to the next page
        offset_count += 200
        
    # Return Tuples
#     Box_office_db = pd.DataFrame([rankings, titles, lifetime_gross, year, URL]).transpose()
#     Box_office_db.columns = (['Lifetime_Gross_Rank', 'Movie_Title', 'Lifetime_Gross', 'Movie_Year', 'URL'])
#     Box_office_db.set_index('Lifetime_Gross_Rank', inplace = True)
    Box_office_tuple_list = []
    for i in range(0,len(rankings)):
        Box_office_tuple_list.append((rankings[i], titles[i], lifetime_gross[i], year[i], URL[i]))
    return Box_office_tuple_list

In [12]:
cnx = mysql.connector.connect(
    host = "ms-proj-db.ckhgdgqfsqpw.us-east-2.rds.amazonaws.com",
    user = "admin",
    passwd = "12345678",
    database = DB_NAME
)
cursor = cnx.cursor()

In [13]:
def add_budget():
        add_business = ("INSERT INTO budget "
                   "(Lifetime_Gross_Rank, Movie_Title, Lifetime_Gross, Movie_Year, URL) "
                   "VALUES (%s, %s, %s, %s, %s)")

        cursor.executemany(add_business, Pull_Box_Office_Revenues())
        cnx.commit()

In [14]:
add_budget()

KeyboardInterrupt: 

In [48]:
def Pull_Movie_Information():    
    # Set fixed values
    PBOR_tuples = Pull_Box_Office_Revenues()
    URL_counter = 0
    URL_List = []
    while URL_counter < len(PBOR_tuples):
        URL_List.append(PBOR_tuples[URL_counter][4])
        URL_counter += 1
    
    # Set counter variable
    offset_count = 1
    
    # Set dummy variables
    rank = []
    title = []
    worldwide_revenue = []
    domestic_distributor = []
    budget = []
    MPAA = []
    running_time = []
    genres = []
    
    # Loop through pages
    for URL in URL_List:
        
        # Set URL for each page equal to response
        response = requests.get(URL)
        
        # Pause the loop
        sleep(randint(1,5))
        
        # Parse the content using BeautifulSoup
        Box_office_soup = BeautifulSoup(response.content, 'html.parser')
        
        # Create find_all if class is a-section a-spacing-none
        find_all = Box_office_soup.find_all(class_='a-section a-spacing-none')
        
        # Add values to the rank list
        rank.append(offset_count)
        
        # Add values to the title list
        title.append([r.text for r in find_all][0].split('(')[0])
        
        # Add values to the worldwide revenue list (different class so need new find_all)
        worldwide_find_all = Box_office_soup.find_all(class_='money')
        worldwide_revenue.append([int(r.text[1:].replace(",", "")) for r in worldwide_find_all][2])
        
        # Add values to the domestic_distributor list
        my_count = 0
        if [r.text for r in find_all][4][:20] == 'Domestic Distributor':
            domestic_distributor.append([r.text for r in find_all][4][20:-30])
        else:
            domestic_distributor.append(None)
            my_count += -1
        
        # Add values to the budget list
 
        if [r.text for r in find_all][6+my_count][0:6] == 'Budget':
                budget.append(int([r.text for r in find_all][6+my_count][7:].replace(',', '')))
        else:
            budget.append(None)
            my_count += -1
        
        # Add values to MPAA list
        if [r.text for r in find_all][8+my_count][0:4] == 'MPAA':
            MPAA.append([r.text for r in find_all][8+my_count][4:])
        else:
            MPAA.append(None)
            my_count += -1
        
        # Add values to running_time list
        if len([r.text for r in find_all][9+my_count].split(" ")) == 5:
            if [r.text for r in find_all][9+my_count][0:12] == 'Running Time':
                running_time.append(int([r.text for r in find_all][9+my_count].split(" ")[1][-1])*60+int([r.text for r in find_all][9+my_count].split(" ")[3]))
            else:
                running_time.append(None)
                my_count += -1
        else:
            if [r.text for r in find_all][9+my_count][0:12] == 'Running Time':
                running_time.append(int([r.text for r in find_all][9+my_count].split(" ")[1][-1])*60)
            else:
                running_time.append(None)
                my_count += -1
        # Add values to genre list/
        counter = 0
        list_of_genres = []
        genre = [r.text for r in find_all]
        while counter < len(genre[10+my_count].split('\n')):
            if counter == 0:
                list_of_genres.append(genre[10+my_count].split('\n')[0][6:])
            else:
                list_of_genres.append(genre[10+my_count].split('\n')[counter].split(' ')[-1])
            counter += 2
        genres.append(list_of_genres)
        
        # Move on to the next page
        print(offset_count)
        offset_count += 1
        
    # Create the Movie Information Tuples
#     Movie_Information_db = pd.DataFrame([rank, title, worldwide_revenue, domestic_distributor, budget, MPAA, running_time, genres]).transpose()
#     Movie_Information_db.columns = (['Rank', 'Title', 'Lifetime Gross Worldwide', 'Domestic Distributor', 'Budget', 'MPAA', 'Running Time (minutes)', 'Genres'])
#     Movie_Information_db.set_index('Rank', inplace = True)
#     return Movie_Information_db        
    Movie_Information_Tuple_List = []
    for i in range(0,len(rank)):
        Movie_Information_Tuple_List.append((rank[i], title[i], worldwide_revenue[i], domestic_distributor[i], budget[i], MPAA[i], running_time[i], genres[i]))
    return Movie_Information_Tuple_List

In [49]:
cnx = mysql.connector.connect(
    host = "ms-proj-db.ckhgdgqfsqpw.us-east-2.rds.amazonaws.com",
    user = "admin",
    passwd = "12345678",
    database = DB_NAME
)
cursor = cnx.cursor()

In [50]:
movie_info = Pull_Movie_Information()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [84]:
movie_info = [list(ele) for ele in movie_info]
for movie in movie_info:
    new_genre = ""
    for genre in movie[-1]:
        new_genre += genre
    movie[-1] = new_genre
movie_info = [tuple(ele) for ele in movie_info_2]

movie_info = [list(ele) for ele in movie_info]
for movie in movie_info:
    movie[-1] = re.sub(r"(\w)([A-Z])", r"\1 \2", movie[-1])
movie_info = [tuple(ele) for ele in movie_info]
movie_info



[(1,
  'Avengers: Endgame ',
  2797800564,
  'Walt Disney Studios Motion Pictures',
  356000000,
  'PG-13',
  181,
  'Action Adventure Drama Sci-Fi'),
 (2,
  'Avatar ',
  2790439000,
  'Twentieth Century Fox',
  237000000,
  'PG-13',
  162,
  'Action Adventure Fantasy Sci-Fi'),
 (3,
  'Titanic ',
  2194439542,
  'Paramount Pictures',
  200000000,
  'PG-13',
  194,
  'Drama Romance'),
 (4,
  'Star Wars: Episode VII - The Force Awakens ',
  2068223624,
  'Walt Disney Studios Motion Pictures',
  245000000,
  'PG-13',
  138,
  'Action Adventure Sci-Fi'),
 (5,
  'Avengers: Infinity War ',
  2048359754,
  'Walt Disney Studios Motion Pictures',
  None,
  'PG-13',
  149,
  'Action Adventure Sci-Fi'),
 (6,
  'Jurassic World ',
  1670400637,
  'Universal Pictures',
  150000000,
  'PG-13',
  124,
  'Action Adventure Sci-Fi'),
 (7,
  'The Lion King ',
  1656943394,
  'Walt Disney Studios Motion Pictures',
  260000000,
  'PG',
  118,
  'Adventure Animation Drama Family Musical'),
 (8,
  'The Avenge

In [85]:
def add_summary():
        add_business = ("INSERT INTO summary "
                   "(Lifetime_Gross_Rank, Movie_Title, Lifetime_Gross, Domestic_Distributor, Budget, MPAA, Running_Time_Minutes, Genres) "
                   "VALUES (%s, %s, %s, %s, %s, %s, %s, %s)")

        cursor.executemany(add_business, movie_info)
        cnx.commit()

In [86]:
add_summary()